In [2]:
#imports here

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import re
import pandas as pd

### This is where the fun begins

In [7]:
#create a function to change the date column into a datetime series
MONTH_TO_NUM = {"JAN" : "01", "FEV" : "02", "MAR" : "03", "AVR" : "04", "MAI" : "05", "JUN" : "06", "JUI" : "07", "AOU" : "08", "SEP" : "09", "OCT" : "10", "NOV" : "11", "DEC" : "12"}

def create_datetime(df, file_name):
    year= file_name[:4]
    df["date"] = pd.to_datetime(year + df["date"].str[-3:].map(MONTH_TO_NUM) + df["date"].str[:2])
    if(file_name[5:7] == "01"):
        #mask used to 
        mask = df["date"].dt.month == 12
        df["date"][mask] -= pd.DateOffset(years = 1)
    return df

In [10]:
#initalize all values here:
#capitals as per naming convention for python when you want to assign a constant
#padding needed to not read repetative noise
OFFSET = 6
#height in pixels of each row
HEIGHT = 27
#config for the ocr config arg
CONFIG = "--psm 12 --oem 1"
#collect the year for the files later by regular expression but for the time being, since we already know that the numbers are in the range of 0-9 position, we'll just slice

#set up paths
SOURCE_PATH = "data/unprocessed/debit/"
SAVE_PATH = "data/processed/debit/"
#-1 to remove the "/"
file_names = os.listdir(SOURCE_PATH)
for file in file_names:
    #create vars to hold each column (we will zip em together later to turn into a dataframe)
    #also reset values
    date = []
    description = []
    charge =[]
    total = []

    #here we will assume that the pdf size is 1700 to 2200 (later we will return to add more adaptability)
    #set up base crop dimensision
    left = 325
    top = 397
    right = 1650
    #bottom will be under constant change per loop, so we'll just initalize it
    bottom = top + HEIGHT

    #come back later to allow for more than one page to be processed
    image = convert_from_path(SOURCE_PATH + file)[1]

    #make shift do-while loop
    #we shall read all the way to the bottom, cleaning will be done in the next steps
    #we could switch it for a forloop sure by dividing the remainging length by the total rows height and just seeing how many times we can run it, but I already wrote this
    while bottom <= 2200:
        #Crop the image to just read row by row
        seg = image.crop((left, top, right, bottom))

        #Lets OCR this thing
        info = pytesseract.image_to_string(seg, config=CONFIG)
        print(info, file)
        info = info.rstrip("\n")
        info = info.split("\n\n")

        #some in precosion error handeling
        if len(info) > 3:
            date.append(info[0])
            description.append(info[1])
            charge.append(info[2])
            total.append(info[3])

        #reset the crop values
        top = bottom + OFFSET
        bottom = top + HEIGHT
    
    #turn into a df
    file_name = file[:10]
    df = pd.DataFrame(list(zip(date, description, charge, total)), columns=["date", "description", "charge", "total"])

    #here we clean the data in all the csvs

    #create a mask to filter out invalid data based on invalid dates (potentially remove and turn into a method to be used in other functions)
    mask = df["date"].str.contains(r"[0-3][0-9]\s*[A-Z]{3}")
    #remove exess
    df= df[mask]
    #To make sure we only have the formate we wish to use
    df['date'] = df["date"].replace(r"[^[0-3][0-9]\s*[A-Z]{3}]", "", regex=True)

    #change the format of total and charge so we can use it without error later
    df["total"] = df["total"].replace(r"[^\d]", "", regex=True)
    df["charge"] = df["charge"].replace(r"[^\d]", "", regex=True)
    #turn it into a decimal
    df["total"] = df["total"].str[:-2] + "." + df["total"].str[-2:]
    df["charge"] = df["charge"].str[:-2] + "." + df["charge"].str[-2:]

    #add datetime:
    df = create_datetime(df,file)
    #index set to false as it is not needed
    df.to_csv(SAVE_PATH + file_name + ".csv", index=False)





01 NOV

FR TENUE DE CPTE

2,50

-2,50
 2016-11-01_Statement.pdf
01 NOV

FRAIS FIXES

0,44

-2,94
 2016-11-01_Statement.pdf
ys OA

NAN

SOA
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
| Sommaire des frais

valeurs ($)
 2016-11-01_Statement.pdf
Ep TONIIE Ac CDTC

sy on
 2016-11-01_Statement.pdf
om nap ee nme anne

aA
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
| Total des frais
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
IPSE | o) UE LIGA liUlis

Tost
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2016-11-01_Statement.pdf
 2

C:\Users\jibz1\AppData\Local\Temp\ipykernel_12820\817675294.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"][mask] -= pd.DateOffset(years = 1)


01 FEV

PRET ETUDIANT AUT PRET BNC

1 300,0

1 309,00
 2017-02-01_Statement.pdf
01 FEV

FRAIS FIXES

2,75

1 306,25
 2017-02-01_Statement.pdf
ys 7c

4 3NnK An

4 °N¢ 95
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
| Sommaire des frais

valeurs ($)
 2017-02-01_Statement.pdf
ED AIC NE DBA ANNIE

ys 7c
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
| Total des trails

2/5
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
_

og

a

7

aman me
 2017-02-01_Statement.pdf
Ne REE RE RP

helen &

eliteetiaieled witcha tet A
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pdf
 2017-02-01_Statement.pd

C:\Users\jibz1\AppData\Local\Temp\ipykernel_12820\817675294.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"][mask] -= pd.DateOffset(years = 1)


02 JAN

C/ACHAT 8421868 COUCHE-TARD # 1

119,99

2 164,38
 2018-02-01_Statement.pdf
03 JAN

G/DEPOT 8421868 JOUR-NUIT

508,64

2 673,02
 2018-02-01_Statement.pdf
03 JAN

C/ACHAT 8421868 THE HOME DEPOT

10,29

2 662,73
 2018-02-01_Statement.pdf
08 JAN

C/ACHAT 8421868 ANIMAL EXPERT

17,24

2 645,49
 2018-02-01_Statement.pdf
08 JAN

C/ACHAT 8421868 ECONOMAX #53

458,75

2 186,74
 2018-02-01_Statement.pdf
08 JAN

C/ACHAT 8421868 REST EGGSQUIS G

17,12

2 169,62
 2018-02-01_Statement.pdf
08 JAN

C/ACHAT 8421868 DOLLARAMA #176

8.05

2 161,57
 2018-02-01_Statement.pdf
08 JAN

C/ACHAT 8421868 BEST BUY #82

80,69

2 080,88
 2018-02-01_Statement.pdf
10 JAN

C/ACHAT 8421 868 PIZZAHUT R296-

30,34

2 050,54
 2018-02-01_Statement.pdf
12 JAN

C/ACHAT 8421868 REST EGGSQUIS G

29,75

2 020,79
 2018-02-01_Statement.pdf
12 JAN

C/ACHAT 8421868 MAXI #8911

85,97

1 934,82
 2018-02-01_Statement.pdf
15 JAN

DEPOT MOBILE

710,22

2 645,04
 2018-02-01_Statement.pdf
17 JAN

C/ACHAT 8421868 EB GAMES 1 866

7

KeyboardInterrupt: 

To note over five years of data, only three anomolies showed up and one page where a second page was given to store more transaction information <br>
solution for anomoly: all three had the same 

In [10]:
#here we will put all the csvs into one big file to play around with
CSV_PATH = "data/processed/debit"
files = os.listdir(CSV_PATH)
#read all files as csvs and put into a list
files_to_concat = [pd.read_csv(CSV_PATH+'/'+file) for file in files]

main_df = pd.concat(files_to_concat)
main_df

,date,description,charge,total
0,01 NOV,FR TENUE DE CPTE,"2,50","-2,50"
1,01 NOV,FRAIS FIXES,"0,44","-2,94"
0,01 DEC,FR TENUE DE CPTE,"2,50","-5,44"
1,01 DEC,FRAIS FIXES,"2,75","-8,19"
2,01 DEC,INTERETS DECOUVERT,"0,05","-8,24"
...,...,...,...,...
11,22 MAR,G/DEPOT 8421868 JOUR-NUIT,"100,00","1 832,62"
12,28 MAR,MOBILE VIREMENTMC,"728,92","1 103,70"
13,29 MAR,COMPTE DIVERS PAYPAL,"44,43","1 059,27"
14,01 AVR,VIREMENT INTERAC,"976,67","2 035,94"


Due to time restraints I had to fix some files by hand so that the values I'd use for the model would be at their most accurate

In [ ]:
#here we merge all csv's into one big